In [2]:

import os
import json
import functools
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
df = pd.read_csv(r"D:\shalini\test_file.csv")

In [4]:
def is_date_column(col_name):
    date_formats = ['%d-%m-%Y']
    for fmt in date_formats:
        try:
            pd.to_datetime(col_name, format=fmt)
            return True
        except ValueError:
            continue  # Try the next format
    return False

# Filter columns with date format 'm/d/Y'
date_columns = [col for col in df.columns if is_date_column(col)]
print(date_columns)

['29-08-2024', '30-08-2024', '31-08-2024']


In [5]:
#ios_rows = df[(df["operating_system_type"] == "Apple iOS")]
ios_rows = df[(df["operating_system_type"] == "Apple iOS") & (df["parent_id_old"].notna()) & (df["parent_id_old"] != '')]
ios_rows.head()

,metric_id,operating_system_type,display_names,hierarchy_id,parent_id_old,parent_id,29-08-2024,30-08-2024,31-08-2024
1,184,Apple iOS,Started Troubleshooting (Solution Center),Apple iOS_SH_1.1,Apple iOS_SH_1,183.0,80,104,112
2,185,Apple iOS,Saw a Result,Apple iOS_SH_1.1.1,Apple iOS_SH_1.1,184.0,76,100,110
3,186,Apple iOS,1st common reason (Unknown),Apple iOS_SH_1.1.1.1,Apple iOS_SH_1.1.1,185.0,60,84,89


In [6]:
#android_rows = df[(df["operating_system_type"] == "Google Android")]
android_rows = df[(df["operating_system_type"] == "Google Android") & (df["parent_id_old"].notna()) & (df["parent_id_old"] != '')]
android_rows.head()

,metric_id,operating_system_type,display_names,hierarchy_id,parent_id_old,parent_id,29-08-2024,30-08-2024,31-08-2024
5,418,Google Android,Started Troubleshooting (Solution Center),Google Android_SH_1.1,Google Android_SH_1,417.0,46,51,45
6,419,Google Android,Saw a Result,Google Android_SH_1.1.1,Google Android_SH_1.1,418.0,46,44,43
7,420,Google Android,1st common reason (Unknown),Google Android_SH_1.1.1.1,Google Android_SH_1.1.1,419.0,36,32,34


In [9]:
ios_sums = ios_rows[date_columns].replace(0, pd.NA)
ios_sums_old_index = ios_sums.index 
print(ios_sums)

   29-08-2024  30-08-2024  31-08-2024
1          80         104         112
2          76         100         110
3          60          84          89


In [11]:
android_sums = android_rows[date_columns].replace(0, pd.NA)
android_sums_old_index = android_sums.index 
print(android_sums)

   29-08-2024  30-08-2024  31-08-2024
5          46          51          45
6          46          44          43
7          36          32          34


In [12]:
ios_sums = ios_sums.reset_index(drop=True)
android_sums= android_sums.reset_index(drop=True)
numerator = ios_sums + android_sums
print(numerator)

   29-08-2024  30-08-2024  31-08-2024
0         126         155         157
1         122         144         153
2          96         116         123


In [13]:
# Filter rows where parent_id_old is present in hierarchy_id
filtered_data = df[
    df['hierarchy_id'].isin(df['parent_id_old'])
]
filtered_data.head(10)

,metric_id,operating_system_type,display_names,hierarchy_id,parent_id_old,parent_id,29-08-2024,30-08-2024,31-08-2024
0,183,Apple iOS,Devices Page Visits,Apple iOS_SH_1,NaN,NaN,3444,3530,3271
1,184,Apple iOS,Started Troubleshooting (Solution Center),Apple iOS_SH_1.1,Apple iOS_SH_1,183.0,80,104,112
2,185,Apple iOS,Saw a Result,Apple iOS_SH_1.1.1,Apple iOS_SH_1.1,184.0,76,100,110
4,417,Google Android,Devices Page Visits,Google Android_SH_1,NaN,NaN,1699,1780,1587
5,418,Google Android,Started Troubleshooting (Solution Center),Google Android_SH_1.1,Google Android_SH_1,417.0,46,51,45
6,419,Google Android,Saw a Result,Google Android_SH_1.1.1,Google Android_SH_1.1,418.0,46,44,43


In [14]:
# Separate Apple iOS and Google Android rows
ios_data = filtered_data[filtered_data['operating_system_type'] == 'Apple iOS']
ios_data.head()

,metric_id,operating_system_type,display_names,hierarchy_id,parent_id_old,parent_id,29-08-2024,30-08-2024,31-08-2024
0,183,Apple iOS,Devices Page Visits,Apple iOS_SH_1,NaN,NaN,3444,3530,3271
1,184,Apple iOS,Started Troubleshooting (Solution Center),Apple iOS_SH_1.1,Apple iOS_SH_1,183.0,80,104,112
2,185,Apple iOS,Saw a Result,Apple iOS_SH_1.1.1,Apple iOS_SH_1.1,184.0,76,100,110


In [15]:
android_data = filtered_data[filtered_data['operating_system_type'] == 'Google Android']
android_data.head()

,metric_id,operating_system_type,display_names,hierarchy_id,parent_id_old,parent_id,29-08-2024,30-08-2024,31-08-2024
4,417,Google Android,Devices Page Visits,Google Android_SH_1,NaN,NaN,1699,1780,1587
5,418,Google Android,Started Troubleshooting (Solution Center),Google Android_SH_1.1,Google Android_SH_1,417.0,46,51,45
6,419,Google Android,Saw a Result,Google Android_SH_1.1.1,Google Android_SH_1.1,418.0,46,44,43


In [16]:
ios_sums = ios_data[date_columns].reset_index(drop=True)
print(ios_sums)

   29-08-2024  30-08-2024  31-08-2024
0        3444        3530        3271
1          80         104         112
2          76         100         110


In [17]:
android_sums = android_data[date_columns].reset_index(drop=True)
print(android_sums)

   29-08-2024  30-08-2024  31-08-2024
0        1699        1780        1587
1          46          51          45
2          46          44          43


In [18]:
denominator = ios_sums + android_sums
print(denominator)

   29-08-2024  30-08-2024  31-08-2024
0        5143        5310        4858
1         126         155         157
2         122         144         153


In [19]:
result = (numerator / denominator) * 100
print(result)

   29-08-2024  30-08-2024  31-08-2024
0    2.449932    2.919021    3.231783
1   96.825397   92.903226   97.452229
2   78.688525   80.555556   80.392157


In [123]:
def calculate_both_test(df, datecolumns, metricid, parentid):
    # Ensure these columns exist in the DataFrame
    if metricid not in df.columns or parentid not in df.columns:
        raise ValueError(f"'{metricid}' or '{parentid}' columns not found in DataFrame.")
    
    # Initialize an empty list to store the new rows
    new_rows = []
    
    # Iterate through each row to generate new rows
    for idx, row in df.iterrows():
        # Check if 'parent_id_old' is None, NaN, or empty
        if pd.isna(row[parentid]) or row[parentid] == None or row[parentid] == '':
            # Handle the case where parent_id_old is missing (second function logic)
            display_name = row['display_names']
            display_name_df = df[df['display_names'] == display_name]
            
            # Create a new row for 'Both' operating system type
            new_row = row.to_dict()
            
            # Sum the date columns for iOS and Android
            for date_column in datecolumns:  # Assuming date columns are passed in datecolumns
                ios_sum = display_name_df[display_name_df['operating_system_type'] == 'Apple iOS'][date_column].sum()
                android_sum = display_name_df[display_name_df['operating_system_type'] == 'Google Android'][date_column].sum()
                
                # Add the sum of both iOS and Android
                new_row[date_column] = ios_sum + android_sum
                
            new_row['operating_system_type'] = 'Both'
            
            # Append the new row to the list of new rows
            new_rows.append(new_row)
        
        else:
            # Handle the case where parent_id_old is not missing (first function logic)
            ios_rows = df[(df["operating_system_type"] == "Apple iOS") & (df["parent_id_old"].notna()) & (df["parent_id_old"] != '')]
            android_rows = df[(df["operating_system_type"] == "Google Android") & (df["parent_id_old"].notna()) & (df["parent_id_old"] != '')]
            
            # Reset indices for consistency
            ios_sums = ios_rows[datecolumns].replace(0, pd.NA).reset_index(drop=True)
            android_sums = android_rows[datecolumns].replace(0, pd.NA).reset_index(drop=True)
            
            # Compute numerator (iOS + Android sums)
            numerator = ios_sums + android_sums

            # Filter rows where parent_id_old is present in hierarchy_id
            filtered_data = df[df['hierarchy_id'].isin(df['parent_id_old'])]
            
            # Separate Apple iOS and Google Android rows in the filtered data
            ios_data = filtered_data[filtered_data['operating_system_type'] == 'Apple iOS']
            android_data = filtered_data[filtered_data['operating_system_type'] == 'Google Android']
            
            # Reset indices for consistency
            ios_sums = ios_data[datecolumns].reset_index(drop=True)
            android_sums = android_data[datecolumns].reset_index(drop=True)
            
            # Compute denominator (iOS + Android sums)
            denominator = ios_sums + android_sums
            
            # Calculate result for each date column
            new_row = row.to_dict()
            for date_col in datecolumns:
                # Perform row-wise division for each date column
                numerator_value = numerator[date_col]
                denominator_value = denominator[date_col]
                print(numerator_value)
                print(denominator_value)
                result = (numerator_value / denominator_value) * 100
                print(result)
                new_row[date_col] = round(result, 3)

                # Avoid division by zero: if denominator is zero, set result to 0
                #if denominator_value != 0:
                #    result = (numerator_value / denominator_value) * 100
                #else:
                #    result = 0  # If denominator is zero, set result to 0
                #print(result)
                #new_row[date_col] = round(result, 3)
            
            # Set the operating_system_type to "Both"
            new_row['operating_system_type'] = 'Both'
            
            # Add the new row to the list of new rows
            new_rows.append(new_row)

    # Convert the list of new rows into a DataFrame
    new_rows_df = pd.DataFrame(new_rows)
    
    # If the second logic was executed, drop duplicates
    new_rows_df = new_rows_df.drop_duplicates(subset=['display_names', 'operating_system_type'], keep='first')
    
    # Append the new rows to the original DataFrame
    df = pd.concat([df, new_rows_df], ignore_index=True)

    return new_rows_df


In [124]:
both_test_df = calculate_both_test(df, date_columns, 'hierarchy_id', 'parent_id_old')
#print(both_test_df)
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 500)      # Show 100 rows
both_test_df.head(100)

0    126
1    122
2     96
Name: 29-08-2024, dtype: int64
0    5143
1     126
2     122
Name: 29-08-2024, dtype: int64
0     2.449932
1    96.825397
2    78.688525
Name: 29-08-2024, dtype: float64
0    155
1    144
2    116
Name: 30-08-2024, dtype: int64
0    5310
1     155
2     144
Name: 30-08-2024, dtype: int64
0     2.919021
1    92.903226
2    80.555556
Name: 30-08-2024, dtype: float64
0    157
1    153
2    123
Name: 31-08-2024, dtype: int64
0    4858
1     157
2     153
Name: 31-08-2024, dtype: int64
0     3.231783
1    97.452229
2    80.392157
Name: 31-08-2024, dtype: float64
0    126
1    122
2     96
Name: 29-08-2024, dtype: int64
0    5143
1     126
2     122
Name: 29-08-2024, dtype: int64
0     2.449932
1    96.825397
2    78.688525
Name: 29-08-2024, dtype: float64
0    155
1    144
2    116
Name: 30-08-2024, dtype: int64
0    5310
1     155
2     144
Name: 30-08-2024, dtype: int64
0     2.919021
1    92.903226
2    80.555556
Name: 30-08-2024, dtype: float64
0    157
1    1

,metric_id,operating_system_type,display_names,hierarchy_id,parent_id_old,parent_id,29-08-2024,30-08-2024,31-08-2024
0,183,Both,Devices Page Visits,Apple iOS_SH_1,NaN,NaN,5143,5310,4858
1,184,Both,Started Troubleshooting (Solution Center),Apple iOS_SH_1.1,Apple iOS_SH_1,183.0,0 2.450 1 96.825 2 78.689 Name: 29-0...,0 2.919 1 92.903 2 80.556 Name: 30-0...,0 3.232 1 97.452 2 80.392 Name: 31-0...
2,185,Both,Saw a Result,Apple iOS_SH_1.1.1,Apple iOS_SH_1.1,184.0,0 2.450 1 96.825 2 78.689 Name: 29-0...,0 2.919 1 92.903 2 80.556 Name: 30-0...,0 3.232 1 97.452 2 80.392 Name: 31-0...
3,186,Both,1st common reason (Unknown),Apple iOS_SH_1.1.1.1,Apple iOS_SH_1.1.1,185.0,0 2.450 1 96.825 2 78.689 Name: 29-0...,0 2.919 1 92.903 2 80.556 Name: 30-0...,0 3.232 1 97.452 2 80.392 Name: 31-0...


In [283]:
def process_csv_data(df, hierarchy_col, parent_id_col):
    """
    Processes the dataframe to generate new rows based on the given logic.
    
    Args:
        df (pd.DataFrame): The input dataframe.
        hierarchy_col (str): The column name for hierarchy ID.
        parent_id_col (str): The column name for parent ID.
        
    Returns:
        pd.DataFrame: The updated dataframe with the generated rows.
    """
    # Create a list for new rows and identify date columns
    new_rows = []
    date_cols = [col for col in df.columns if "2024" in col]  # Identify date columns
    sequence_number = 1

    ios_df = df[(df["operating_system_type"] == "Apple iOS")]
    android_df = df[(df["operating_system_type"] == "Google Android")]

    # Iterate over unique display_names
    for display_name in df["display_names"].unique():
        # Filter for Apple iOS and Google Android rows for this display_name
        ios_rows = df[(df["display_names"] == display_name) & 
                      (df["operating_system_type"] == "Apple iOS")]
        android_rows = df[(df["display_names"] == display_name) & 
                          (df["operating_system_type"] == "Google Android")]
        
        # Check if the rows exist
        if ios_rows.empty or android_rows.empty:
            continue
        
        # Sum the date values for numerator
        ios_sums = ios_rows[date_cols].replace(0, pd.NA).sum()
        android_sums = android_rows[date_cols].replace(0, pd.NA).sum()
        numerator = ios_sums + android_sums
        #print(numerator)
        
        # Determine parent_id_old status and calculate denominator if needed
        for _, ios_row in df.iterrows():
            if pd.isna(ios_row["parent_id_old"]):  # Case: parent_id_old is null/empty
                result = numerator
            else:  # Case: parent_id_old is not null/empty
                parent_hierarchy_id = ios_row["parent_id_old"]
                

                # Filter the rows where 'operating_system_type' is "Apple iOS" and 'parent_id_old' is not null or empty
                ios_rows = df[(df["operating_system_type"] == "Apple iOS") & (df["parent_id_old"].notna()) & (df["parent_id_old"] != '')]

                # Filter the rows where 'operating_system_type' is "Google Android" and 'parent_id_old' is not null or empty
                android_rows = df[(df["operating_system_type"] == "Google Android") & (df["parent_id_old"].notna()) & (df["parent_id_old"] != '')]

                ios_sums = ios_rows[date_cols].replace(0, pd.NA)
                android_sums = android_rows[date_cols].replace(0, pd.NA)
                ios_sums = ios_sums.reset_index(drop=True)
                android_sums= android_sums.reset_index(drop=True)
              
                numerator = ios_sums + android_sums
                #print(numerator)


                






                filtered_data = df[df['hierarchy_id'].isin([parent_hierarchy_id])]
                ios_data = filtered_data[filtered_data['operating_system_type'] == 'Apple iOS']
                android_data = filtered_data[filtered_data['operating_system_type'] == 'Google Android']
                ios_sums = ios_data[date_cols].reset_index(drop=True)
                #print(ios_sums)
                android_sums = android_data[date_columns].reset_index(drop=True)
                #print(android_sums)
                denominator = ios_sums + android_sums
                print(denominator)
                
                result = (numerator / denominator) * 100
                #print(result)

                """
                # Retrieve parent rows using the same display_name filter
                parent_ios_row = ios_df[(ios_df[hierarchy_col] == parent_hierarchy_id)]
                parent_android_row = android_df[(android_df[hierarchy_col] == parent_hierarchy_id)]
                
                # Sum values while excluding zeros
                
                parent_ios_sums = parent_ios_row[date_cols].replace(0, pd.NA).sum()
                parent_android_sums = parent_android_row[date_cols].replace(0, pd.NA).sum()
                print(parent_ios_sums)
                print(parent_android_sums)
                
                
                #denominator = parent_ios_sums + parent_android_sums
                #print(denominator)
                
                
                # Ensure denominator is valid
                if denominator > 0:
                    result = (numerator + denominator) * 100
                else:
                    continue  # Skip if denominator is invalid
                print(result)
                
            # Generate new row
            new_row = {
                "display_names": display_name,
                "metric_id": f"Both_ID_{sequence_number}",
                "hierarchy_id": None,
                "parent_id_old": None,
                "parent_id": None,
                **result.to_dict()  # Add date column values
            }
            new_rows.append(new_row)
            sequence_number += 1
    
    # Append new rows to the original dataframe
    new_df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    return new_df
    """
